In [1]:
import pandas as pd
import numpy as np

In [2]:
#import data
#suffix = 'base'
suffix = 'amtk_current'

gen = pd.read_csv(f'./inputs/inputs_gen_{suffix}.csv')
load = pd.read_csv(f'./inputs/inputs_load_{suffix}.csv')
rencf = pd.read_csv(f'./inputs/inputs_renewableCF.csv')
trans = pd.read_csv(f'./inputs/inputs_trans_{suffix}.csv')

print(load)

        Unnamed: 0         r     t  demandLoad
0                0  ERC_REST     1    33450.05
1                1  ERC_REST     2    33443.15
2                2  ERC_REST     3    33842.20
3                3  ERC_REST     4    34583.95
4                4  ERC_REST     5    35800.65
...            ...       ...   ...         ...
534294      534294   WECC_WY  8755     1431.00
534295      534295   WECC_WY  8756     1405.00
534296      534296   WECC_WY  8757     1355.00
534297      534297   WECC_WY  8758     1283.00
534298      534298   WECC_WY  8759     1270.00

[534299 rows x 4 columns]


In [3]:
#create a list of our regions
regions = set(load['r'])

In [4]:
#calculate regions where generation > load
for reg in regions:
    gn = np.zeros((8759,))
    gn += gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] != 'Solar').values * (gen['FuelType'] != 'Wind').values * (gen['FuelType'] != 'wind_generator').values * (gen['FuelType'] != 'solar_generator').values, 'Capacity'].values.sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Solar').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'solarCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'solar_generator').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'solarCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Wind').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'windCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'wind_generator').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'windCF'].values).sum()

    if (load.loc[load['r'] == reg, 'demandLoad'] > gn).values.sum() > 0:
        print(f'Generation cannot match load in region {reg}')
        print(max(load.loc[load['r'] == reg, 'demandLoad'] - gn))
    

Generation cannot match load in region NY_Z_J
442.9857609751016


In [5]:
#find the total generation for the region where generation > load
 
region = 'NY_Z_J' ## select regions from the above
t = 5080 

print( 'Nuclear generation ' + str(gen.loc[gen['RegionName'] == region].loc[gen['FuelType'] == 'Nuclear','Capacity'].sum()))

print( 'Total generation ' + str(gen.loc[gen['RegionName'] == region,'Capacity'].sum()))

#look at the transmission into the region where generation > load

print( 'Transmission into the region ' + str(trans.loc[(trans['r2'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()*.972))

print( 'Transmission out of the region ' + str(trans.loc[(trans['r1'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()))

#find the total amount of energy needed to/from that region if all used

print( f'Maximum net energy left after use from the region at time {t} ' + 
    str(trans.loc[(trans['r2'] == region).values, 'transCap'].sum()*.972 + 
       gen.loc[gen['RegionName'] == region,'Capacity'].sum() - 
        trans.loc[(trans['r1'] == region).values, 'transCap'].sum() - 
       load.loc[(load['r'] == 'NY_Z_GI')].loc[(load['t'] == t)]['demandLoad'].values))

Nuclear generation 0.0
Total generation 9599.0
Transmission into the region 10432.476
Transmission out of the region 5575
Maximum net energy left after use from the region at time 5080 [11833.98461542]


In [6]:
#find what time stamps are associated with demand > capacity 
#(we will go back and manually change our load based on this, assuming some of the region blacks out)
load.loc[load['r'] == region].loc[load['t'].isin(range(5001,6000))].loc[load['demandLoad'] >= gen.loc[gen['RegionName'] == region,'Capacity'].sum()]

,Unnamed: 0,r,t,demandLoad
188945,188945,NY_Z_J,5007,9600.389745
188946,188946,NY_Z_J,5008,9715.389745
188947,188947,NY_Z_J,5009,9683.389745
189013,189013,NY_Z_J,5075,9826.389745
189014,189014,NY_Z_J,5076,10085.389745
189015,189015,NY_Z_J,5077,10262.389745
189016,189016,NY_Z_J,5078,10364.389745
189017,189017,NY_Z_J,5079,10432.389745
189018,189018,NY_Z_J,5080,10359.389745
189019,189019,NY_Z_J,5081,10222.389745


In [7]:
print( 'Solar generation ' + str(gen.loc[gen['FuelType'] == 'Solar','Capacity'].sum()))

print( 'Wind generation ' + str(gen.loc[gen['FuelType'] == 'Wind','Capacity'].sum()))


Solar generation 10842.0
Wind generation 68058.0


In [8]:
(gen.loc[gen['FuelType'] == 'Solar','Capacity'].sum() + gen.loc[gen['FuelType'] == 'solar_generator','Capacity'].sum())/1000

np.float64(10.842)